In [1]:
!pip install numpy<2
!pip install transformers torch
from transformers import pipeline
import pandas as pd
from tqdm import tqdm

zsh:1: no such file or directory: 2


/Users/ruobing/opt/anaconda3/envs/DS5110/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
classifier = pipeline(
    "zero-shot-classification",
    model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli"
)

labels = ["Mild", "Moderate", "Severe"]
template = "After reviewing the incident, the most appropriate classification for emotional and psychological impact is {}."

df = pd.read_csv("../all_merged_cleaned_standardized.csv")

text_columns = [
    "assault_desc", "response_action", "contributing_factors",
    "nature_of_injury", "injury_location"
]
for col in text_columns:
    df[col] = df[col].fillna("")

def create_context(row):
    return (
        f"Incident Description: {row['assault_desc']} "
        f"Response Action Taken: {row['response_action']}. "
        f"Contributing Factors: {row['contributing_factors']}. "
        f"Nature of Injury: {row['nature_of_injury']}. "
        f"Injury Location: {row['injury_location']}."
    )

df["full_context"] = df.apply(create_context, axis=1)

predictions = []
for text in tqdm(df["full_context"], desc="Predicting emotional impact"):
    if not text.strip():
        predictions.append("None")
        continue
    result = classifier(text, labels, hypothesis_template=template, multi_label=False)
    predictions.append(result["labels"][0])

df["emotional_impact_ML"] = predictions
df.to_csv("all_merged_labeled_contextual.csv", index=False)

Device set to use mps:0
Predicting emotional impact: 100%|██████████| 1341/1341 [05:48<00:00,  3.85it/s]


In [3]:
df.head(20)

,event_time,victim_profession,department,perpetrator_type,violence_type,emotional_impact,assault_desc,response_action,contributing_factors,physical_injury_level,...,charges_pressed,law_enforcement_contacted,nature_of_injury,injury_location,source_file,source_sheet,severity,source_data,full_context,emotional_impact_ML
0,2025-01-12 00:00:00,other,cadillac 2 north,employee,physical,,A patient was seen touching a staff member on ...,Reported to department leader,Patient - Lack of Compliance/Adherence,No,...,No,No,Inappropriate touching,buttocks,NorthernLight,January 2025 NLH,Mild,df1,Incident Description: A patient was seen touch...,Moderate
1,2025-01-14 00:00:00,tech,cadillac 2 south,employee,verbal,,Patient in 201 had a significant early this mo...,Patient Behavior Safety Flag/Care Plan initiated,Action by Patient/Resident; Altered Mental Sta...,Unknown,...,NaN,,,,NorthernLight,January 2025 NLH,NaN,df1,Incident Description: Patient in 201 had a sig...,Severe
2,2025-01-14 00:00:00,admin,grounds & common areas (outside),employee,verbal,,It was reported that on the afternoon outdoor ...,De-escalation techniques,Action by Patient/Resident; Communication Failure,No,...,NaN,,,,NorthernLight,January 2025 NLH,Mild,df1,Incident Description: It was reported that on ...,Mild
3,2025-01-04 00:00:00,tech,katahdin 1 south,employee,physical,,this patient was escalated due to a game of mo...,De-escalation techniques; EAP - Offered/Employ...,Action by Patient/Resident; Task - Inexperienc...,Unknown,...,NaN,,,,NorthernLight,January 2025 NLH,NaN,df1,Incident Description: this patient was escalat...,Moderate
4,2025-01-15 00:00:00,tech,katahdin 2 north,employee,physical,,Patient continued to show poor boundaries with...,De-escalation techniques; Reported to departme...,Patient - Lack of Compliance/Adherence; Patien...,No,...,NaN,,,,NorthernLight,January 2025 NLH,Mild,df1,Incident Description: Patient continued to sho...,Moderate
5,2025-01-05 00:00:00,tech,katahdin 2 south,employee,physical,,Cart was brought onto unit with two liter soda...,No Action,Action by Patient/Resident,Yes,...,NaN,,Abrasion,Right temple,NorthernLight,January 2025 NLH,Moderate,df1,Incident Description: Cart was brought onto un...,Severe
6,2025-01-05 00:00:00,tech,katahdin 2 south,employee,physical,,During an escalation/escort a patient kicked t...,No Action,Action by Patient/Resident,Yes,...,NaN,,Other,Right knee,NorthernLight,January 2025 NLH,Moderate,df1,Incident Description: During an escalation/esc...,Mild
7,2025-01-15 00:00:00,tech,katahdin 2 south,employee,physical,3 staff affected,"PT punched psych tech in the back, then advanc...",No Action,Action by Patient/Resident; Agitated; Willful ...,Unknown,...,NaN,,,,NorthernLight,January 2025 NLH,NaN,df1,Incident Description: PT punched psych tech in...,Mild
8,2025-01-15 00:00:00,nurse,katahdin 2 south,employee,physical,,Patient spit in staff's face and eye during a ...,Physical Restraints utilized; Treatment Provided,Action by Patient/Resident,Yes,...,NaN,,Exposure - Body Fluids,eyes/face/mouth/nose,NorthernLight,January 2025 NLH,Moderate,df1,Incident Description: Patient spit in staff's ...,Moderate
9,2025-01-03 00:00:00,tech,katahdin 2 south,employee,physical,,"pt squirted hand sanitizer in my eye, i was ab...",No Action,Action by Patient/Resident; Agitated,No,...,NaN,,,,NorthernLight,January 2025 NLH,Mild,df1,Incident Description: pt squirted hand sanitiz...,Moderate
